<a href="https://colab.research.google.com/github/mitsuoxv/erp/blob/master/you_can_try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# You can let the presidents speak

I download texts.

In [1]:
!wget https://raw.githubusercontent.com/mitsuoxv/erp/master/texts/presidents/{1947..2020}_pres.txt --directory-prefix=/tmp/

--2020-07-17 12:09:44--  https://raw.githubusercontent.com/mitsuoxv/erp/master/texts/presidents/1947_pres.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3322 (3.2K) [text/plain]
Saving to: ‘/tmp/1947_pres.txt’

1947_pres.txt       100%[===================>]   3.24K  --.-KB/s    in 0s      

2020-07-17 12:09:45 (54.8 MB/s) - ‘/tmp/1947_pres.txt’ saved [3322/3322]

--2020-07-17 12:09:45--  https://raw.githubusercontent.com/mitsuoxv/erp/master/texts/presidents/1948_pres.txt
Reusing existing connection to raw.githubusercontent.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 25989 (25K) [text/plain]
Saving to: ‘/tmp/1948_pres.txt’

1948_pres.txt       100%[===================>]  25.38K  --.-KB/s    in 0.01s   

2020-07-17 12:09:45 (1.85 MB/s) - ‘/

I download the model.

In [2]:
!wget https://raw.githubusercontent.com/mitsuoxv/erp/master/ErpPresModel.h5 --directory-prefix=/tmp/

--2020-07-17 12:10:19--  https://raw.githubusercontent.com/mitsuoxv/erp/master/ErpPresModel.h5
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10855044 (10M) [application/octet-stream]
Saving to: ‘/tmp/ErpPresModel.h5.2’

ErpPresModel.h5.2   100%[===================>]  10.35M  34.9MB/s    in 0.3s    

2020-07-17 12:10:20 (34.9 MB/s) - ‘/tmp/ErpPresModel.h5.2’ saved [10855044/10855044]



In [4]:
corpus = []

for year in range(1947, 2020):
    input_filename = '/tmp/{}_pres.txt'.format(year)
    
    with open(input_filename, 'r') as f:
        text = f.read()
    
    corpus = corpus + text.lower().split('\n')

In [5]:
import re

corpus = [line for line in corpus if not re.fullmatch(r'^.$', line)]
corpus = [line for line in corpus if not re.fullmatch(r'^\(.\)$', line)]
corpus = [line for line in corpus if not re.fullmatch(r'^.\.$', line)]

corpus = [line for line in corpus if line != ""]

corpus = [re.sub('—', ' ', line) for line in corpus]
corpus = [re.sub('“', '', line) for line in corpus]
corpus = [re.sub('”', '', line) for line in corpus]

In [6]:
corpus = [line for line in corpus if len(re.split('[ —]', line)) < 35]

In [7]:
import tensorflow as tf

tf.__version__

'2.2.0'

In [8]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np

In [9]:
tokenizer = Tokenizer()

tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

In [10]:
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]

ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [11]:
model = Sequential()
model.add(Embedding(total_words, 64, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(20)))
model.add(Dense(total_words, activation='softmax'))

In [12]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

I load the saved model.

In [13]:
model.load_weights('/tmp/ErpPresModel.h5')

You can write seed text here.

In [14]:
seed_text = "This Nation has high aspirations"
next_words = 30

In [15]:
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict_classes(token_list, verbose=0)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
This Nation has high aspirations and to the lowest of the year reaching up about 1 2 percent of gnp and the extraordinarily high level of output and prices and real official tends to trigger
